## Tokenization

Tokenization is at the heart of much weirdness of the LLMs.

    1. Why can't LLM spell words?
    2. Why can't LLM do super simple string processing tasks like reversing a string?
    3. Why is LLM worse aat non-English languages (e.g. Japanese)?
    4. Why is LLM bad at simple arithmetic?
    5. Why did GPT-2 have more than necessary trouble coding in Python?
    5. Why does my LLM abruptly halt when it sees the string "<|endoftext|>"?
    6. What is this weird warning I get about a "trailing whitespace"?
    7. Why does the LLM break if I ask it about "SolidGoldMagikarp"?
    8. Why should I prefer to use YAML over JSON with LLMs?
    9. Why is LLM not actually end-to-end language modeling?
    10. What is the real root of suffering?

#### Tokenization

In [3]:
"안녕하세요 👋 (hello in Korean)"

'안녕하세요 👋 (hello in Korean)'

In [10]:
[ord(x) for x in "안녕하세요 👋 (hello in Korean)"]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 41]

In [14]:
#encoding with utf-8

list("안녕하세요 👋 (hello in Korean)".encode("utf-8"))


[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 41]

In [15]:
!git init

Initialized empty Git repository in C:/Users/anish/OneDrive/Desktop/LLM Projects/GPT_Tokenizer/tokenizer_notebook/.git/


In [17]:
!git remote add origin https://github.com/AnishPaudyal/GPT-Tokenizer.git

error: remote origin already exists.


In [18]:
!git add .

In [19]:
!git commit -m "Intro to tokenization + UTF-8 encoding (why we cannot use raw bytes as tokens)"

[master (root-commit) b7f30c3] Intro to tokenization + UTF-8 encoding (why we cannot use raw bytes as tokens)
 2 files changed, 213 insertions(+)
 create mode 100644 .ipynb_checkpoints/Tokenization-checkpoint.ipynb
 create mode 100644 Tokenization.ipynb


In [24]:
!git push -u origin master

branch 'master' set up to track 'origin/master'.


Everything up-to-date


### byte-pair encoding algorithm


In [15]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8") #raw bytes (single string byte)
tokens = list(map(int, tokens))
print(text)
print("\n")
#text length
print("Text length:", len(text))
print("\n")
print(tokens)
print("\n")
#token length
print("Token length:", len(tokens))

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.


Text length: 533


[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 14

In [28]:
def get_pair(ids):
    count = {}
    for pair in zip(ids, ids[1:]):
        count[pair] = count.get(pair, 0) + 1
    return count

frequent_pairs = get_pair(tokens)
# print(frequent_pairs)
print(sorted(((v,k) for k,v in frequent_pairs.items()), reverse = True))

[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

In [30]:
top_pair = max(frequent_pairs, key = stats.get)
top_pair

(101, 32)

In [ ]:
!git commit -m "Generated Raw bytes (